In [35]:
import pandas as pd
import re
import os

import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, udf, lit, explode, split, regexp_extract, col, isnan, isnull, desc, when, sum, to_date, desc, regexp_replace, count, to_timestamp
from pyspark.sql.types import IntegerType, TimestampType

In [36]:
#setting visualization options
# https://www.1week4.com/it/machine-learning/udacity-data-engineering-capstone-project/
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

# modify visualization of the notebook, for easier view
from IPython.core.display import display, HTML
display(HTML("""<style> p { max-width:90% !important; } h1 {font-size:2rem!important } h2 {font-size:1.6rem!important } 
h3 {font-size:1.4rem!important } h4 {font-size:1.3rem!important }h5 {font-size:1.2rem!important }h6 {font-size:1.1rem!important }</style>"""))# Do all imports and installs here


In [37]:
def create_spark_session():
    """
    This function creates a Spark Sesson and includes necessary Jar and adoop packages in the configuration. 
    """
#     spark = SparkSession \
#         .builder \
#         .getOrCreate()

    spark=SparkSession \
    .builder \
    .config("spark.jars.repositories", "https://repos.spark-packages.org/") \
    .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11") \
    .enableHiveSupport() \
    .getOrCreate()
    return spark

# def create_spark_session():
#     """
#     This function creates a Spark Sesson and includes necessary Jar and adoop packages in the configuration. 
#     """
#     spark = SparkSession \
#         .builder \
#         .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
#         .getOrCreate()
#     return spark



In [30]:
spark = create_spark_session()

In [38]:
I94_DATASET_PATH = '../../../../../data/18-83510-I94-Data-2016/'

filelist = os.listdir(I94_DATASET_PATH)
print(f"The dataset contains {len(filelist)} files")

The dataset contains 12 files


In [39]:
for file in filelist:
    size = os.path.getsize('{}/{}'.format(I94_DATASET_PATH, file))
    print(f'{file} - dim(bytes): {size}')

i94_apr16_sub.sas7bdat - dim(bytes): 471990272
i94_sep16_sub.sas7bdat - dim(bytes): 569180160
i94_nov16_sub.sas7bdat - dim(bytes): 444334080
i94_mar16_sub.sas7bdat - dim(bytes): 481296384
i94_jun16_sub.sas7bdat - dim(bytes): 716570624
i94_aug16_sub.sas7bdat - dim(bytes): 625541120
i94_may16_sub.sas7bdat - dim(bytes): 525008896
i94_jan16_sub.sas7bdat - dim(bytes): 434176000
i94_oct16_sub.sas7bdat - dim(bytes): 556269568
i94_jul16_sub.sas7bdat - dim(bytes): 650117120
i94_feb16_sub.sas7bdat - dim(bytes): 391905280
i94_dec16_sub.sas7bdat - dim(bytes): 523304960


In [41]:
I94_TEST_FILE = '../../../../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat'

#df_I94 = spark.read.format('com.github.saurfang.sas.spark').load(I94_TEST_FILE).persist()

df = spark.read.load(I94_TEST_FILE)

Py4JJavaError: An error occurred while calling o347.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:290)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:538)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:611)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:603)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:801)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(R

In [ ]:
df_I94.limit(5).toPandas().head()

In [ ]:
df_I94.printSchema()

#### Convert doubles to integers

In [ ]:
# Snippet taken from https://www.1week4.com/it/machine-learning/udacity-data-engineering-capstone-project/
toInt = udf(lambda x: int(x) if x!=None else x, IntegerType())

for colname, coltype in df_I94.dtypes:
    if coltype == 'double':
        df_I94 = df_I94.withColumn(colname, toInt(colname))

#### Convert strings to dates

In [ ]:
df_I94 = df_I94.withColumn('dtadfile',to_date((col("dtadfile")),"yyyyMMdd"))

In [ ]:
df_I94 = df_I94.withColumn('dtaddto',to_date(col("dtaddto"),"MMddyyyy"))

#### Convert SAS epoch dates from integers to dates

In [ ]:
# from https://knowledge.udacity.com/questions/66798
from datetime import datetime, timedelta
from pyspark.sql import types as T
def convert_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return None
udf_datetime_from_sas = udf(lambda x: convert_datetime(x), T.DateType())

In [ ]:
df_I94.limit(5).toPandas().head()

In [ ]:
# df_I94 = df_I94.withColumn('dtaddto',to_date(col("dtaddto"),"MMddyyyy")
df_I94 = df_I94.withColumn("arrdate", udf_datetime_from_sas("arrdate")) 

In [ ]:
df_I94.limit(5).toPandas().head()

In [ ]:
df_I94 = df_I94.withColumn("depdate", udf_datetime_from_sas("depdate")) 

In [ ]:
df_I94.limit(5).toPandas().head()

In [ ]:
df_I94.printSchema()

In [ ]:
df_I94.select("gender").groupBy("gender").count().show()

In [ ]:
df_I94.select("airline").groupBy("airline").count().show()

In [ ]:
df_I94.select("airline").groupBy("airline").count().show()

In [ ]:
df_I94.select("arrDate").groupBy("arrDate").count().show()

#### Get max and min arrDates

In [ ]:
df_I94.select("arrDate").groupBy("arrDate").max().show()

#### Get max and min depDates

#### Count and delete duplicates

#### Find nulls

#### Deal with Nulls

In [77]:
df_I94.limit(5).toPandas().head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrDate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6,2016,4,692,692,XXX,None,NaN,None,None,37,2,1,None,None,None,T,None,U,None,1979,2016-10-28,None,None,None,1897628485,None,B2
1,7,2016,4,254,276,ATL,None,1.0,AL,None,25,3,1,2013-08-11,SEO,None,G,None,Y,None,1991,None,M,None,None,-558170966,00296,F1
2,15,2016,4,101,101,WAS,None,1.0,MI,None,55,2,1,2016-04-01,None,None,T,O,None,M,1961,2016-09-30,M,None,OS,666643185,93,B2
3,16,2016,4,101,101,NYC,None,1.0,MA,None,28,2,1,2016-04-01,None,None,O,O,None,M,1988,2016-09-30,None,None,AA,-2020819182,00199,B2
4,17,2016,4,101,101,NYC,None,1.0,MA,None,4,2,1,2016-04-01,None,None,O,O,None,M,2012,2016-09-30,None,None,AA,-2020817382,00199,B2
